# С помощью бертов решить задачу классификации

## Импорты

In [1]:
import numpy as np
import pandas as pd

import torch
import torch_directml
from torch import argmax
from torch.nn.functional import softmax
from transformers import BertTokenizerFast, AutoModelForSequenceClassification

for i in range(torch_directml.device_count()):
    print(f'{i}) {torch_directml.device_name(i)}')

0) Radeon RX 590 Series


## Настройки

In [2]:
DEVICE = 0
MODEL = 'blanchefort/rubert-base-cased-sentiment'

device = 'cpu'
if type(DEVICE) is int:
    device = torch_directml.device(DEVICE)
else:
    device = DEVICE

## Модель

In [3]:
class SentimentClassifier:
    CALSSES = ['Нейтральный', 'Позитивный', 'Негативный']
    
    def __init__(self, model, device='cpu'):
        self._tokenizer = BertTokenizerFast.from_pretrained(model)
        self._model = AutoModelForSequenceClassification.from_pretrained(model)
        self.to(device)
    
    @torch.no_grad()
    def predict(self, text, aggregate=True):
        inputs = self._tokenizer(text,
                                 max_length=512,
                                 padding=True,
                                 truncation=True,
                                 return_tensors='pt')

        inputs = inputs.to(self._device)
        outputs = self._model(**inputs)
        predicted = softmax(outputs.logits, dim=1)
        predicted = argmax(predicted, dim=1).to('cpu').numpy()
        
        if aggregate:
            try:
                return self.CALSSES[predicted[0]]
            except:
                return 'Ошибка определения класса!'
        
        return predicted
    
    def to(self, device):
        self._model = self._model.to(device)
        self._device = device

classifier = SentimentClassifier(MODEL, device=device)

In [4]:
print(classifier.predict('Я обожаю тяжелую музыку!'))

Позитивный


In [5]:
print(classifier.predict('Прекрасный пример полного отсутствия мозга.'))

Негативный


In [6]:
print(classifier.predict('Скоро мы будем делать чат бота.'))

Нейтральный
